In [2]:
import pandas as pd
import numpy as np
import math 

Read the book sent over by Westley and Stephanie

In [34]:
#dat_orig = pd.read_excel("20190624_sales_fcst.xlsx", header = 2)
dat_orig = pd.read_excel("20190808_sales.xlsx", header = 0)
dat_orig.fillna(method='ffill', inplace=True)
dat_orig = dat_orig.filter(regex=r'(Wk.|Unnamed: 4|Report in thousands|Unnamed: 0)')
dat_orig.head(50)

,Unnamed: 0,Report in thousands,Report in thousands.1,Unnamed: 4,Feb Wk. 1 2019,Feb Wk. 2 2019,Feb Wk. 3 2019,Feb Wk. 4 2019,Mar Wk. 5 2019,Mar Wk. 6 2019,...,Nov Wk. 43 2019,Dec Wk. 44 2019,Dec Wk. 45 2019,Dec Wk. 46 2019,Dec Wk. 47 2019,Dec Wk. 48 2019,Jan Wk. 49 2019,Jan Wk. 50 2019,Jan Wk. 51 2019,Jan Wk. 52 2019
0,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,5.476461e+06,4.662842e+06,5.506713e+06,7.207304e+06,6.706429e+06,7.941871e+06,...,9.629377e+06,8.453331e+06,8.315628e+06,9.078337e+06,6.376926e+06,4.936864e+06,5.794772e+06,4.361261e+06,4.705942e+06,4.954138e+06
1,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Forecast,6.573989e+06,5.102038e+06,6.248079e+06,8.703870e+06,7.677581e+06,7.629826e+06,...,9.629377e+06,8.453331e+06,8.315628e+06,9.078337e+06,6.376926e+06,4.936864e+06,5.794772e+06,4.361261e+06,4.705942e+06,4.954138e+06
2,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Plan,6.649002e+06,5.416112e+06,6.519510e+06,8.560530e+06,8.344701e+06,8.645828e+06,...,1.007645e+07,9.025481e+06,8.938418e+06,9.903067e+06,6.790026e+06,5.497583e+06,4.303109e+06,6.030907e+06,5.316052e+06,4.894777e+06
3,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Var to Plan,-1.172541e+06,-7.532700e+05,-1.012797e+06,-1.353226e+06,-1.638272e+06,-7.039573e+05,...,-4.470694e+05,-5.721499e+05,-6.227896e+05,-8.247297e+05,-4.130997e+05,-5.607191e+05,1.491663e+06,-1.669646e+06,-6.101102e+05,5.936121e+04
4,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Target,5.476461e+06,4.662842e+06,5.506713e+06,7.207304e+06,6.706429e+06,7.941871e+06,...,9.974831e+06,8.863771e+06,8.676071e+06,9.352461e+06,6.542920e+06,5.089523e+06,5.814496e+06,4.335850e+06,4.652347e+06,5.392389e+06
5,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),LY,5.169145e+06,5.444805e+06,6.017282e+06,7.693575e+06,9.149243e+06,7.047698e+06,...,8.529283e+06,8.181361e+06,8.196857e+06,9.226034e+06,6.719577e+06,5.130113e+06,5.742053e+06,4.322338e+06,4.355694e+06,4.680824e+06
6,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),LLY,5.043022e+06,4.763425e+06,6.154673e+06,7.483757e+06,8.109833e+06,6.191291e+06,...,8.242003e+06,8.083615e+06,8.209195e+06,8.741188e+06,5.848454e+06,4.263754e+06,5.560383e+06,4.723382e+06,4.438348e+06,5.201560e+06
7,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Actual to Plan %,-1.763484e-01,-1.390795e-01,-1.553486e-01,-1.580774e-01,-1.963249e-01,-8.142162e-02,...,-4.436776e-02,-6.339273e-02,-6.967559e-02,-8.328023e-02,-6.083919e-02,-1.019937e-01,3.466478e-01,-2.768483e-01,-1.147675e-01,1.212746e-02
8,Retail,Sls Total Rtl,Retail Sales (Stores & Concierge),Actual to LY %,5.945214e-02,-1.436164e-01,-8.485047e-02,-6.320481e-02,-2.669964e-01,1.268744e-01,...,1.289784e-01,3.324266e-02,1.448984e-02,-1.600867e-02,-5.099287e-02,-3.766965e-02,9.181167e-03,9.005058e-03,8.041141e-02,5.839010e-02
9,Direct,Sls Total Rtl,Direct Total Sales,Actual (TY) / Forecast,4.089837e+06,3.747693e+06,2.806511e+06,6.487625e+06,4.296148e+06,5.890494e+06,...,9.025120e+06,1.270649e+07,5.769346e+06,3.571574e+06,3.601724e+06,3.995767e+06,5.676268e+06,3.052906e+06,3.126105e+06,3.334049e+06


In [23]:
h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
h2 = [x+1 for x in h1]

months = h1 + h2
weeks = list(range(1, 53))

month_map = pd.DataFrame()
month_map['FISCAL_MO'] = months
month_map['FISCAL_WK'] = weeks



In [47]:
def format_sales(df, history):
    
    ids = list(df.iloc[:, np.r_[0:1, 2:4]].columns)
    vals = list(df.iloc[:,4:].columns)
    
    melt = pd.melt(dat_orig, id_vars = ids, value_vars = vals, var_name = 'FISCAL_WK', value_name = 'SALES')
    
    melt.rename(columns={'Unnamed: 0':'Channel',
                      'Report in thousands.1': 'sales_segment',
                          'Unnamed: 4':'summation_type'
                          }, 
                 inplace=True)
    
    #sub = melt[melt['summation_type'] == "Actual (TY) / Forecast"]
    sub = melt[melt['summation_type'] == history]
    sub = sub[sub['sales_segment'].isin(['Direct Total Sales', 'Retail Sales (Stores & Concierge)'])]
    sub[['Channel', 'SALES']].groupby('Channel').agg('sum')
    
    sub.FISCAL_WK = sub.FISCAL_WK.str[8:]
    sub.FISCAL_WK = sub.FISCAL_WK.str[:2]
    sub.FISCAL_WK = sub.FISCAL_WK.str.strip()
    
    h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
    h2 = [x+6 for x in h1]

    months = h1 + h2
    weeks = list(range(1, 53))
    qtrs = newList = [math.ceil(x / 3) for x in months]

    month_map = pd.DataFrame()
    month_map['FISCAL_MO'] = months
    month_map['FISCAL_WK'] = weeks
    month_map['FISCAL_QTR'] = qtrs
    
    sub.FISCAL_WK = sub.FISCAL_WK.astype(str).astype(int)
    sales_fcst = pd.merge(sub, month_map, on='FISCAL_WK', how='inner')
    #fcst = sales_fcst.query('summation_type == "Actual (TY) / Forecast"')
    fcst = sales_fcst[sales_fcst['summation_type'] == history]
    
    fcst.Channel = np.where(fcst['Channel']=='Direct', 'D', 'R')
    
    return fcst

In [48]:
fcst = format_sales(dat_orig, "Actual (TY) / Forecast")
fcst.head()

,Channel,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO,FISCAL_QTR
0,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1,1
1,D,Direct Total Sales,Actual (TY) / Forecast,1,4089837.30,1,1
2,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1,1
3,D,Direct Total Sales,Actual (TY) / Forecast,2,3747693.15,1,1
4,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1,1


In [49]:
fcst.SALES.sum()

675793728.8557029

In [50]:
ly = format_sales(dat_orig, "LY")
ly.head()

,Channel,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO,FISCAL_QTR
0,R,Retail Sales (Stores & Concierge),LY,1,5169144.62,1,1
1,D,Direct Total Sales,LY,1,3037794.40,1,1
2,R,Retail Sales (Stores & Concierge),LY,2,5444805.20,1,1
3,D,Direct Total Sales,LY,2,4395773.70,1,1
4,R,Retail Sales (Stores & Concierge),LY,3,6017282.22,1,1


In [51]:
ly.SALES.sum()

672492208.5799999

In [78]:
ly[['SALES', 'FISCAL_QTR']].groupby('FISCAL_QTR').agg('sum')

,SALES
FISCAL_QTR,
1,1.751628e+08
2,1.678821e+08
3,1.684904e+08
4,1.609570e+08


In [54]:
fcst[['SALES', 'FISCAL_QTR']].groupby('FISCAL_QTR').agg('sum')

,SALES
FISCAL_QTR,
1,1.733971e+08
2,1.690767e+08
3,1.707190e+08
4,1.626009e+08


How did dollars per customer shake out this year compared to last year? We are .166% comp in sales to LY

In [59]:
q2_ly = ly.query('FISCAL_QTR <= 2')['SALES'].agg('sum')
q2_ty = fcst.query('FISCAL_QTR <= 2')['SALES'].agg('sum')

In [61]:
((q2_ly - q2_ty) / q2_ly)*100

0.16644478289808703

In [79]:
ly[['SALES', 'FISCAL_MO']].groupby('FISCAL_MO').agg('sum')

,SALES
FISCAL_MO,
1,40480928.64
2,74050667.69
3,60631159.04
4,65813323.87
5,60535198.47
6,41533538.75
7,40944325.03
8,70509940.00
9,57036135.46


In [80]:
fcst[['SALES', 'FISCAL_MO']].groupby('FISCAL_MO').agg('sum')

,SALES
FISCAL_MO,
1,3.998499e+07
2,7.070443e+07
3,6.270773e+07
4,6.315664e+07
5,6.082445e+07
6,4.509561e+07
7,3.981289e+07
8,7.353905e+07
9,5.736708e+07


In [19]:
fcst.to_pickle('month_2019')

In [67]:
mth_data = pd.read_csv('monthly_sales_to_custs.csv').sort_values('Channel')
mth_data.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,FISCAL_MO,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust
0,2018,D,1.397545e+07,130751,1,1,5.689090e+05,6912,0,0.040708,0.052864,5.470167e+05,6489,NTB_1,1,5.782865e+05,6688,0.945927,0.970245,84.299070
69,2018,D,2.103208e+07,207167,4,11,1.743798e+07,163431,1,0.829114,0.788885,4.129484e+06,18955,Existing__Omni,_Omni,1.890481e+07,149634,0.218436,0.126676,217.857257
68,2018,D,2.103208e+07,207167,4,11,1.743798e+07,163431,1,0.829114,0.788885,1.477533e+07,130679,Existing_1,1,1.890481e+07,149634,0.781564,0.873324,113.065800
67,2018,D,2.512522e+07,223096,4,10,2.085962e+07,174849,1,0.830226,0.783739,3.822122e+06,15022,Existing__Omni,_Omni,2.172181e+07,158738,0.175958,0.094634,254.434973
66,2018,D,2.512522e+07,223096,4,10,2.085962e+07,174849,1,0.830226,0.783739,1.789968e+07,143716,Existing_1,1,2.172181e+07,158738,0.824042,0.905366,124.548993


In [62]:
fcst.head()

,Channel,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO,FISCAL_QTR
0,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1,1
1,D,Direct Total Sales,Actual (TY) / Forecast,1,4089837.30,1,1
2,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1,1
3,D,Direct Total Sales,Actual (TY) / Forecast,2,3747693.15,1,1
4,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1,1


In [63]:
direct_total = fcst[['Channel', 'SALES', 'FISCAL_MO']].query('Channel == "D"').groupby(['FISCAL_MO', 'Channel']).agg('sum').reset_index()
retail_total = fcst[['Channel', 'SALES', 'FISCAL_MO']].query('Channel == "R"').groupby(['FISCAL_MO', 'Channel']).agg('sum').reset_index()
mth_sales = pd.concat([direct_total, retail_total])

In [64]:
mth_sales.SALES.sum()

675793728.8557029

In [65]:
mth_sales = mth_sales.rename(index=str, columns={"SALES": "channel_mth_sales"})

In [68]:
mth_dat = pd.merge(mth_data, mth_sales, on = ['FISCAL_MO', 'Channel'], how = 'inner')
mth_dat.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,FISCAL_MO,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,...,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_mth_sales
0,2018,D,1.397545e+07,130751,1,1,5.689090e+05,6912,0,0.040708,...,5.470167e+05,6489,NTB_1,1,5.782865e+05,6688,0.945927,0.970245,84.299070,17131665.6
1,2018,D,1.397545e+07,130751,1,1,1.183314e+06,12971,2,0.084671,...,1.135112e+06,12172,React_1,1,1.220619e+06,12606,0.929948,0.965572,93.255990,17131665.6
2,2018,D,1.397545e+07,130751,1,1,1.183314e+06,12971,2,0.084671,...,8.550678e+04,434,React__Omni,_Omni,1.220619e+06,12606,0.070052,0.034428,197.020231,17131665.6
3,2018,D,1.397545e+07,130751,1,1,1.222312e+07,110866,1,0.874614,...,1.965367e+06,8327,Existing__Omni,_Omni,1.293655e+07,104400,0.151924,0.079761,236.023360,17131665.6
4,2018,D,1.397545e+07,130751,1,1,1.222312e+07,110866,1,0.874614,...,1.097118e+07,96073,Existing_1,1,1.293655e+07,104400,0.848076,0.920239,114.196292,17131665.6


In [69]:
mth_dat['rec_seg_exp_sales'] = mth_dat.sales_pct * mth_dat.channel_mth_sales * mth_dat.chan_rec_sales_pct
mth_dat.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,FISCAL_MO,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,...,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_mth_sales,rec_seg_exp_sales
0,2018,D,1.397545e+07,130751,1,1,5.689090e+05,6912,0,0.040708,...,6489,NTB_1,1,5.782865e+05,6688,0.945927,0.970245,84.299070,17131665.6,6.596811e+05
1,2018,D,1.397545e+07,130751,1,1,1.183314e+06,12971,2,0.084671,...,12172,React_1,1,1.220619e+06,12606,0.929948,0.965572,93.255990,17131665.6,1.348939e+06
2,2018,D,1.397545e+07,130751,1,1,1.183314e+06,12971,2,0.084671,...,434,React__Omni,_Omni,1.220619e+06,12606,0.070052,0.034428,197.020231,17131665.6,1.016142e+05
3,2018,D,1.397545e+07,130751,1,1,1.222312e+07,110866,1,0.874614,...,8327,Existing__Omni,_Omni,1.293655e+07,104400,0.151924,0.079761,236.023360,17131665.6,2.276362e+06
4,2018,D,1.397545e+07,130751,1,1,1.222312e+07,110866,1,0.874614,...,96073,Existing_1,1,1.293655e+07,104400,0.848076,0.920239,114.196292,17131665.6,1.270723e+07


In [70]:
mth_dat['implied_custs'] = mth_dat.rec_seg_exp_sales / mth_dat.d_p_cust
mth_dat.implied_custs.sum()

4687620.183636964

In [75]:
mth_dat[['FISCAL_MO', 'Channel', 'implied_custs']].groupby(['FISCAL_MO']).agg('sum')

,implied_custs
FISCAL_MO,
1,283073.462088
2,463463.771823
3,386727.361130
4,405336.818592
5,423570.394900
6,386935.872789
7,320755.831577
8,434151.813508
9,340246.592302


In [72]:
omni = year_end_fcst.query('Segment == "_Omni"')
omni

NameError: name 'year_end_fcst' is not defined

In [16]:
noms = pd.merge(omni, omni, on = 'Recency', how='inner')
noms = noms

In [19]:
noms

,Channel_x,Recency,Customer_Type_x,Segment_x,implied_custs_x,Channel_y,Customer_Type_y,Segment_y,implied_custs_y
0,D,0,NTB__Omni,_Omni,20520.120222,D,NTB__Omni,_Omni,20520.120222
1,D,0,NTB__Omni,_Omni,20520.120222,R,NTB__Omni,_Omni,18863.811405
2,R,0,NTB__Omni,_Omni,18863.811405,D,NTB__Omni,_Omni,20520.120222
3,R,0,NTB__Omni,_Omni,18863.811405,R,NTB__Omni,_Omni,18863.811405
4,D,1,Existing__Omni,_Omni,161649.139806,D,Existing__Omni,_Omni,161649.139806
5,D,1,Existing__Omni,_Omni,161649.139806,R,Existing__Omni,_Omni,160016.121752
6,R,1,Existing__Omni,_Omni,160016.121752,D,Existing__Omni,_Omni,161649.139806
7,R,1,Existing__Omni,_Omni,160016.121752,R,Existing__Omni,_Omni,160016.121752
8,D,2,React__Omni,_Omni,40838.961381,D,React__Omni,_Omni,40838.961381
9,D,2,React__Omni,_Omni,40838.961381,R,React__Omni,_Omni,36733.887016


In [37]:
oms = noms.query('Channel_x != Channel_y').copy()
oms['omni'] = oms.implied_custs_x + oms.implied_custs_y
oms = oms[['Customer_Type_x', 'omni']].copy()
oms = oms.drop_duplicates()

In [38]:
oms.omni.sum()

438622.0415827384

In [40]:
oms.head()

,Customer_Type_x,omni
1,NTB__Omni,39383.931627
5,Existing__Omni,321665.261558
9,React__Omni,77572.848397


In [47]:
melt.rename(columns={'Unnamed: 0':'MASTER_CHANNEL',
                      'Report in thousands.1': 'sales_segment',
                          'Unnamed: 4':'summation_type'
                          }, 
                 inplace=True)

We will do a quick aggregation at this point to confirm that the summations we've applied are working properly. We want to verify that these totals line up with the totals taken from Wes's workbook. The grand total should match the current sales forecast.

In [48]:
sub = melt[melt['summation_type'] == "Actual (TY) / Forecast"]
sub = sub[sub['sales_segment'].isin(['Direct Net Sales', 'Retail Sales (Stores & Concierge)'])]
sub.head()
sub[['MASTER_CHANNEL', 'SALES']].groupby('MASTER_CHANNEL').agg('sum')

,SALES
MASTER_CHANNEL,
Direct,2.782062e+08
RETAIL,4.019490e+08


In [35]:
sub.SALES.sum()

680155158.749575

In [36]:
sub.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES
54,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 1 2019,5476461.31
135,Direct,Direct Net Sales,Actual (TY) / Forecast,Feb Wk. 1 2019,4089837.30
243,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 2 2019,4662842.01
324,Direct,Direct Net Sales,Actual (TY) / Forecast,Feb Wk. 2 2019,3747693.15
432,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 3 2019,5506713.02


Now the data is formatted appropriately so we are ready to move with the next step. Ultimately we need our forecasts at the monthly level, so our next task is to clean up the `FISCAL_WK` column so that we can use to to create a `FISCAL_MO` column.

In [49]:
sub.FISCAL_WK = sub.FISCAL_WK.str[8:]
sub.FISCAL_WK = sub.FISCAL_WK.str[:2]
sub.FISCAL_WK = sub.FISCAL_WK.str.strip()
sub.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES
54,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31
135,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30
243,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01
324,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15
432,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02


In [50]:
sub.FISCAL_WK.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52'], dtype=object)

In [51]:
h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
h2 = [x+1 for x in h1]

months = h1 + h2
weeks = list(range(1, 53))

month_map = pd.DataFrame()
month_map['FISCAL_MO'] = months
month_map['FISCAL_WK'] = weeks

month_map.head()

,FISCAL_MO,FISCAL_WK
0,1,1
1,1,2
2,1,3
3,1,4
4,2,5


In [52]:
month_map.shape

(52, 2)

In [53]:
sub.dtypes

MASTER_CHANNEL     object
sales_segment      object
summation_type     object
FISCAL_WK          object
SALES             float64
dtype: object

In [54]:
sub.FISCAL_WK = sub.FISCAL_WK.astype(str).astype(int)
sales_fcst = pd.merge(sub, month_map, on='FISCAL_WK', how='inner')
sales_fcst.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1


In [55]:
sales_fcst.head(26)

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1
5,Direct,Direct Net Sales,Actual (TY) / Forecast,3,2806510.55,1
6,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,4,7207303.66,1
7,Direct,Direct Net Sales,Actual (TY) / Forecast,4,6487624.60,1
8,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,5,6706428.53,2
9,Direct,Direct Net Sales,Actual (TY) / Forecast,5,4296148.16,2


We might want to work with the LY , plan, or other summation types, but for this notebook we are only interested in the `Actual (TY) / Forecast` sales totals. So now we are ready to subset this data to just forecasted sales so we can get to work on building together a customer forecast.

In [58]:
fcst = sales_fcst.query('summation_type == "Actual (TY) / Forecast"')
fcst.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1


In [57]:
fcst.shape

(104, 3)